# NHANES I Survival Model
This is a cox proportional hazards model on data from NHANES I with followup mortality data from the NHANES I Epidemiologic Followup Study. It is designed to illustrate how SHAP values enable the interpretion of XGBoost models with a clarity traditionally only provided by linear models. We see interesting and non-linear patterns in the data, which suggest the potential of this approach. Keep in mind the data has not yet been checked by us for calibrations to current lab tests and so you should not consider the results as actionable medical insights, but rather a proof of concept.

Note that support for Cox loss and SHAP interaction effects were only recently merged, so you will need the latest master version of XGBoost to run this notebook.

**Note 2:** this notebook originally from [here](https://slundberg.github.io/shap/notebooks/NHANES%20I%20Survival%20Model.html)

In [ ]:
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl